In [4]:
# 1- Importar el archivo csv y extraer las variables Total_Score, Study_hours_per_Week y Attendance(%)
# 2- Limpiar datos aplicando al menos un algoritmo de deteccion de outliers
# 3- Normalizar/estandarizar datos
# 4- Agrupamiento K-means
# 5- Etiquetado y descripcion de clusters 

In [5]:
import pandas as pd
datos = pd.read_csv("datos.csv")
print(datos.head)

<bound method NDFrame.head of      Student_ID First_Name Last_Name                       Email  Gender  Age  \
0         S1000       Omar  Williams     student0@university.com  Female   22   
1         S1001      Maria     Brown     student1@university.com    Male   18   
2         S1002      Ahmed     Jones     student2@university.com    Male   24   
3         S1003       Omar  Williams     student3@university.com  Female   24   
4         S1004       John     Smith     student4@university.com  Female   23   
...         ...        ...       ...                         ...     ...  ...   
4995      S5995      Ahmed     Jones  student4995@university.com    Male   19   
4996      S5996       Emma     Brown  student4996@university.com    Male   19   
4997      S5997       John     Brown  student4997@university.com  Female   24   
4998      S5998       Sara     Davis  student4998@university.com    Male   23   
4999      S5999      Maria     Brown  student4999@university.com  Female   21  